# Load Counts and Create Seurat Object

In [ ]:
library(Seurat)
library(Matrix)
library(dplyr)
library(pryr)
library(RColorBrewer)
library(ggplot2)

In [ ]:
read_mtx <- function(folder) {
  counts <- readMM(file.path(folder, "matrix.mtx"))
  genes <- read.table(file.path(folder, "genes.tsv"), header = FALSE, sep = "\t", stringsAsFactors = FALSE)
  barcodes <- read.table(file.path(folder, "barcodes.tsv"), header = FALSE, sep = "\t", stringsAsFactors = FALSE)

  # Assign row and column names
  rownames(counts) <- genes$V2  # Second column contains gene names
  colnames(counts) <- barcodes$V1  # First column contains cell barcodes

  return(counts)
}

WT1_counts <- read_mtx("YOUR_PATH")
WT2_counts <- read_mtx("YOUR_PATH")
Mutant1_counts <- read_mtx("YOUR_PATH")
Mutant2_counts <- read_mtx("YOUR_PATH")


In [ ]:
WT1_seurat <- CreateSeuratObject(WT1_counts, project = "WT1", min.cells = 0, min.features = 0)
WT2_seurat <- CreateSeuratObject(WT2_counts, project = "WT2", min.cells = 0, min.features = 0)
Mutant1_seurat <- CreateSeuratObject(Mutant1_counts, project = "Mutant1", min.cells = 0, min.features = 0)
Mutant2_seurat <- CreateSeuratObject(Mutant2_counts, project = "Mutant2", min.cells = 0, min.features = 0)


rownames(WT1_seurat[["RNA"]]@layers[["counts"]]) <- rownames(WT1_counts)
rownames(WT2_seurat[["RNA"]]@layers[["counts"]]) <- rownames(WT2_counts)
rownames(Mutant1_seurat[["RNA"]]@layers[["counts"]]) <- rownames(Mutant1_counts)
rownames(Mutant2_seurat[["RNA"]]@layers[["counts"]]) <- rownames(Mutant2_counts)

head(rownames(WT1_seurat[["RNA"]]@layers[["counts"]]))


WT1_seurat$Experiment <- "WT1"
WT2_seurat$Experiment <- "WT2"
Mutant1_seurat$Experiment <- "Mutant1"
Mutant2_seurat$Experiment <- "Mutant2"



# Merge all samples into one Seurat object
combined_seurat <- merge(WT1_seurat, y = c(WT2_seurat, Mutant1_seurat, Mutant2_seurat), 
                         add.cell.ids = c("WT1", "WT2", "Mutant1", "Mutant2"), project = "Combined")

rownames(combined_seurat[["RNA"]]@layers[["counts.WT1"]]) <- rownames(WT1_counts)
rownames(combined_seurat[["RNA"]]@layers[["counts.WT2"]]) <- rownames(WT2_counts)
rownames(combined_seurat[["RNA"]]@layers[["counts.Mutant1"]]) <- rownames(Mutant1_counts)
rownames(combined_seurat[["RNA"]]@layers[["counts.Mutant2"]]) <- rownames(Mutant2_counts)


# Filter, Normalize, Scale

In [ ]:
# Filter
combined_seurat <- subset(combined_seurat, 
                          subset = nFeature_RNA > 100 & 
                                   nFeature_RNA < 8000)
# Normalize the data
combined_seurat <- NormalizeData(combined_seurat)

# Find variable features
combined_seurat <- FindVariableFeatures(combined_seurat)

# Scale the data
combined_seurat <- ScaleData(combined_seurat)

# Combine Matrices for Clustering Analysis

In [ ]:
# Ensure matrices are sparse
WT1_counts <- as(WT1_counts, "dgCMatrix")
WT2_counts <- as(WT2_counts, "dgCMatrix")
Mutant1_counts <- as(Mutant1_counts, "dgCMatrix")
Mutant2_counts <- as(Mutant2_counts, "dgCMatrix")

combined_counts_matrix <- RowMergeSparseMatrices(WT1_counts, WT2_counts)

saveRDS(combined_counts_matrix, file = "combined_counts_matrix.rds")


# Perform K Means Clustering

In [ ]:
combined_counts_matrix <- readRDS("combined_counts_matrix.rds")

# Ensure column names match between the matrix and Seurat object
common_cells <- colnames(combined_seurat)  # Cells remaining after filtering
combined_counts_matrix <- combined_counts_matrix[, common_cells, drop = FALSE]  

set.seed(42)
k_clusters <- 11  # Adjust number of clusters if needed
kmeans_result <- kmeans(t(as.matrix(combined_counts_matrix)), centers = k_clusters, nstart = 25)

# Ensure cluster assignments match Seurat object cells
if (length(kmeans_result$cluster) == length(colnames(combined_seurat))) {
    combined_seurat$kmeans_cluster <- as.factor(kmeans_result$cluster)
} else {
    stop("Mismatch in number of cells between clustering results and Seurat object")
}

saveRDS(combined_seurat, file = "k_means_seurat.rds")

# Perform Louvain Clustering

In [ ]:
# Ensure column names match between the matrix and Seurat object AFTER FILTERING
common_cells <- colnames(combined_seurat)  # Cells remaining after filtering
combined_counts_matrix <- combined_counts_matrix[, common_cells, drop = FALSE]

# Remove zero-sum genes and cells (to avoid NaNs in similarity)
valid_genes <- rowSums(combined_counts_matrix) > 0
valid_cells <- colSums(combined_counts_matrix) > 0
combined_counts_matrix <- combined_counts_matrix[valid_genes, valid_cells]

combined_counts_matrix <- as(combined_counts_matrix, "CsparseMatrix")


# Compute column norms (for each cell) instead of row norms
col_norms <- sqrt(colSums(combined_counts_matrix^2))

# Compute cosine similarity between cells instead of genes
cosine_sim <- tcrossprod(t(combined_counts_matrix)) / (col_norms %o% col_norms)


# Replace NaN values with zeros
cosine_sim[is.nan(cosine_sim)] <- 0

# Convert to a distance matrix (1 - similarity)
cosine_dist <- as.matrix(1 - cosine_sim)

# Convert to a graph (now correctly using cell-to-cell distances)
graph <- graph_from_adjacency_matrix(cosine_dist, mode = "undirected", weighted = TRUE, diag = FALSE)

# Run Louvain clustering
clustering <- cluster_louvain(graph)


cluster_membership <- membership(clustering)
names(cluster_membership) <- common_cells

combined_seurat$seurat_clusters <- as.factor(cluster_membership[common_cells])

# Save the Seurat object with clustering results
saveRDS(combined_seurat, file = "seurat_louvain_clusters.rds")
